In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 1.9 MB 15.0 MB/s 
     |████████████████████████████████| 895 kB 70.4 MB/s 
     |████████████████████████████████| 3.3 MB 71.0 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=e5d0e9b364d46aa94774f25f618234474328b107cc11089c8e4e3ced3f122c2a
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 2.6 MB 15.1 MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-py3-none-any.whl size=7053 sha256=fd34c0543b502260bbb906e819f32ff7676e1b099325101aa0deeb35291247fc
  Stored in directory: /root/.cache/pip/wheels/12/40/73/a0765d65e793332b79dfe6c34c713e7c0066ea785191b3f50a
Successfully built language-detector
     |████████████████████████████████| 1.2 MB 14.5 MB/s 
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=e3468d35bf005eb3

True

In [4]:
import tensorflow, sentence_transformers, transformers
tensorflow.__version__, sentence_transformers.__version__, transformers.__version__

('2.6.0', '0.4.1', '4.3.3')

In [5]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [6]:
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex, ContextEncoderConv, ContextEncoderSimple
from src.dataset.ldabert_2 import LDABERT2Dataset

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np
from utils.experiments import get_experiments_json, get_experiments, save_results

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

## Experiment

In [7]:
experiments_config = get_experiments_json('ldabert2_complex_finetune_test')
experiments_config_df = pd.DataFrame.from_dict(experiments_config)
experiments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,max_sentence_length,lstm_size,lstm_dropout_percentage,cnn_filters,cnn_kernel_size,pool_size,gamma,pct_data,augment_pct,bert_trainable,epochs
0,ldabert,clinical,0.25,256,128,256,0.25,8,3,2,15,1,1,True,200
1,ldabert,wiki,0.25,256,128,256,0.25,8,3,2,15,1,1,True,200
2,ldabert,fiction,0.25,256,128,256,0.25,8,3,2,15,1,1,True,200


In [8]:
experiments_config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [9]:
def get_random_hash(k):
  import random, string
  x = ''.join(random.choices(string.ascii_letters + string.digits, k=k))
  return x

In [ ]:
# just do wiki for now
for experiment in experiments_config[1:2]: # wiki
    dataset_type = experiment['dataset_type']
    final_dropout = experiment['final_dropout']
    dense_neurons = experiment['dense_neurons']
    
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    gamma = experiment['gamma']
    max_sentence_length = experiment['max_sentence_length']
    bert_trainable = experiment['bert_trainable']
    finetuning_epochs = 5 # finetune the system for 5 epochs before training with the frozen finetuned BERT weights
    epochs = experiment['epochs']
    BATCH_SIZE = 16
    print("params:", experiment)
    random_hash = get_random_hash(5)
    experiment['epochs'] = epochs

    for step in ["finetune","frozen"]:
    # for step in ["frozen"]:
    
      # init model
      print("initializing model...")
      model = ContextEncoderSimple(final_dropout=final_dropout,
                              dense_neurons=dense_neurons,
                            gamma=gamma,
                            max_sentence_length=max_sentence_length,
                            bert_trainable=True if step == "finetune" else False)
      
      # print("number of params: ", sum([np.prod(keras.get_value(w).shape) for w in model.trainable_weights]))
      
      # init dataset
      print("initializing dataset...")
      dataset = LDABERT2Dataset(dataset_type=dataset_type,
                              pct_data=pct_data,
                            max_seq_length=max_sentence_length,
                              max_segment_length=5,
                              augment_pct=augment_pct,
                              split="train",
                        artificial_segments=True)
      
      # process dataset
      print("processing dataset...")
      sentences, tokenized_sentences, labels = dataset.process()

      vectors_filename = '{}_{}_artificial-segments-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments)

      saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

      if len(saved_vectors) == 0:
          saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

      left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
      lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

      # get class weight
      neg, pos = np.bincount(labels.flatten())
      initial_bias = np.log([pos/neg])
      
      total=len(labels)
      weight_for_0 = (1 / neg)*(total)/2.0 
      weight_for_1 = (1 / pos)*(total)/2.0

      class_weight = {0: weight_for_0, 1: weight_for_1}
      print("class weight", class_weight)
      
      # create checkpoint path
      checkpoint_filepath = '{}/models/LDABERT2/hybrid-conv/{}-{}-{}-pct-{}-aug_{}/checkpoint'.format(
                              config.root_path,
                              dataset.dataset_type,                    
                              len(saved_sentences), 
                              dataset.pct_data,
                              dataset.augment_pct,
                              random_hash)
      
      # continue training
      if step == "frozen":
        # random_hash = 'WLZ0q'
        checkpoint_filepath = '{}/models/LDABERT2/hybrid-conv/{}-{}-{}-pct-{}-aug_{}/checkpoint'.format(
                              config.root_path,
                              dataset.dataset_type,                    
                              len(saved_sentences), 
                              dataset.pct_data,
                              dataset.augment_pct,
                              random_hash)
      
      print(checkpoint_filepath)
      
      # get callbacks ready.
      model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
          filepath=checkpoint_filepath,
          save_weights_only=False,
          monitor='val_accuracy',
          save_best_only=True,
          mode="auto",
          save_freq="epoch")

      early_stopping = tf.keras.callbacks.EarlyStopping(
          monitor='val_accuracy', 
          verbose=1,
          patience=10,
          mode='max',
          restore_best_weights=True)
      
      log_path = '{}/models/LDABERT2/hybrid-conv/{}-{}-{}-pct-{}-aug_{}/checkpoint/training.log'.format(
                              config.root_path,
                              dataset.dataset_type,                    
                              len(saved_sentences), 
                              dataset.pct_data,
                              dataset.augment_pct,
                              random_hash)

      csv_logger = tf.keras.callbacks.CSVLogger(log_path, separator=",", append=False)

      callbacks = [
      #     early_stopping,
          model_checkpoint_callback,
          csv_logger
      ]
      
      # compiling model
      print("compiling the model...")
      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                    loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[
                        keras.metrics.BinaryAccuracy(name='accuracy')
                    ])
      
      try:
          model.load_weights(checkpoint_filepath)
          print("model loaded.")
      except:
          print("No checkpoint available.")
      
      print("starting the training process...")
      # remove warnings
      tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
      history = model.fit([
                              left_input, mid_input, right_input, 
                              lda_left_input, lda_mid_input, lda_right_input
                          ], 
                          tf.convert_to_tensor(saved_labels), 
                          epochs=finetuning_epochs if step == "finetune" else epochs,
                          validation_split=0.25,
                          batch_size=BATCH_SIZE,
                          verbose=1, 
                          class_weight=class_weight,
                          callbacks=callbacks)

      # assigning history to experiment object for saving.
      experiment["history"] = history.history
      experiment["hash"] = random_hash
      
      print("saving results...")
      save_results(experiment)

params: {'bert_type': 'ldabert', 'dataset_type': 'wiki', 'final_dropout': 0.25, 'dense_neurons': 256, 'max_sentence_length': 128, 'lstm_size': 256, 'lstm_dropout_percentage': 0.25, 'cnn_filters': 8, 'cnn_kernel_size': 3, 'pool_size': 2, 'gamma': 15, 'pct_data': 1, 'augment_pct': 1, 'bert_trainable': True, 'epochs': 200}
initializing model...


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


initializing dataset...
processing dataset...
artificial segments True
artificial segments True
class weight {0: 0.5223680536163082, 1: 11.67665418227216}
/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/hybrid-conv/wiki-37412-1-pct-1-aug_pyWLQ/checkpoint
compiling the model...
No checkpoint available.
starting the training process...
Epoch 1/5
1754/1754 [==============================] - 811s 443ms/step - loss: 0.7538 - accuracy: 0.5209 - val_loss: 0.6980 - val_accuracy: 0.5055


...

Epoch 197/200
1754/1754 [==============================] - 321s 183ms/step - loss: 0.6535 - accuracy: 0.5858 - val_loss: 0.5489 - val_accuracy: 0.5859


Epoch 198/200
1754/1754 [==============================] - 321s 183ms/step - loss: 0.6471 - accuracy: 0.5861 - val_loss: 0.6106 - val_accuracy: 0.5862


Epoch 199/200
1754/1754 [==============================] - 322s 184ms/step - loss: 0.6550 - accuracy: 0.5864 - val_loss: 0.6388 - val_accuracy: 0.5865


Epoch 200/200
1754/1754 [==============================] - 322s 183ms/step - loss: 0.6561 - accuracy: 0.5866 - val_loss: 0.6048 - val_accuracy: 0.5867


saving results...


In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored